# Chunking Experiments with MLflow Tracking

This notebook explores different chunking strategies for RAG:
- Multiple chunk sizes (128, 256, 512, 1024 tokens)
- Multiple overlap values (0, 25, 50 tokens)
- Multiple embedders
- Token-based chunking using each embedder's tokenizer

All experiments are tracked in MLflow for comparison.

In [1]:
import numpy as np
import faiss
import torch
import gc
from time import time
from datasets import load_from_disk, disable_caching, Dataset, load_dataset
from rag.config import PROJECT_ROOT
from rag.tracking import ExperimentTracker
from rag.utils import embed_dataset, get_metrics
from rag.embeddings import LocalEmbedder
from rag.config import settings
from rag.ingestion.chunker import RecursiveChunker

/home/ergot/projects/rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load datasets
doc_ds = load_dataset("rag-datasets/rag-mini-bioasq", "text-corpus", split="passages")
doc_ds = doc_ds.filter(lambda row: row['passage'] != 'nan')
query_ds = load_dataset("rag-datasets/rag-mini-bioasq", "question-answer-passages", split="test")

# Precompute
doc_id_to_text = doc_ds.select_columns(['id', 'passage']).to_pandas().set_index('id')['passage'].to_dict()
queries = np.array(query_ds['question'])
qrels = [np.array(eval(gold)) for gold in query_ds['relevant_passage_ids']]
qrels_counts = [len(s) for s in qrels]

disable_caching()

In [3]:
# Initialize tracker
tracker = ExperimentTracker('chunking-experiments-bioasq')

# Experiment configuration
embedder_models = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-MiniLM-L12-v2",
    "BAAI/bge-small-en-v1.5",
    "BAAI/bge-base-en-v1.5",
    "BAAI/bge-large-en-v1.5",
]

chunk_sizes = [None, 128, 256, 512]  # None = no chunking, others in tokens
chunk_overlaps = [0, 25, 50, 100]  # in tokens

faiss_metric = 'IP'
rerank_model = None


def deduplicate_retrieved_docs(retrieved_ids_all, k):
    """
    Deduplicate document IDs per query, keeping only first occurrence.
    
    For chunked retrieval, multiple chunks from same document may be retrieved.
    This keeps only the highest-ranked occurrence of each unique document.
    
    Args:
        retrieved_ids_all: (n_queries, n_retrieved) array of document IDs
        k: Number of unique documents to keep per query
    
    Returns:
        (n_queries, k) array of unique document IDs
    """
    deduped = []
    for query_results in retrieved_ids_all:
        seen = set()
        unique_docs = []
        for doc_id in query_results:
            if doc_id not in seen:
                unique_docs.append(doc_id)
                seen.add(doc_id)
            if len(unique_docs) == k:
                break
        # Pad if needed
        while len(unique_docs) < k:
            unique_docs.append(0)  # Padding with 0 (won't match any real doc)
        deduped.append(unique_docs)
    return np.array(deduped)

[2025-10-26 10:12:58] [rag.tracking] [INFO] Tracking to: http://localhost:5000
[2025-10-26 10:12:58] [rag.tracking] [INFO] Experiment: chunking-experiments-bioasq


In [4]:
# Run experiments
for embedder_name in embedder_models:
    embedder_name_short = embedder_name.split('/')[-1]
    
    for chunk_size in chunk_sizes:
        # Skip overlap experiments when not chunking
        overlaps_to_test = [0] if chunk_size is None else chunk_overlaps
        
        for chunk_overlap in overlaps_to_test:
            # Skip invalid configurations
            if chunk_size is not None and chunk_overlap >= chunk_size:
                continue
            
            print(f"\n{'='*80}")
            print(f"Testing: {embedder_name_short} | chunk_size={chunk_size} | overlap={chunk_overlap}")
            print(f"{'='*80}")
            
            try:
                # Create chunked dataset using token-based chunking
                if chunk_size is None:
                    current_doc_ds = doc_ds
                else:
                    chunker = RecursiveChunker(
                        chunk_size=chunk_size,
                        chunk_overlap=chunk_overlap,
                        embedder_model=embedder_name,
                    )
                    current_doc_ds = chunker.chunk_dataset(doc_ds, text_col='passage', id_col='id')
                    # Rename columns to match expected format
                    current_doc_ds = current_doc_ds.rename_column('doc_id', 'parent_id')
                    current_doc_ds = current_doc_ds.rename_column('text', 'passage')
                    print(f"Created {len(current_doc_ds)} chunks from {len(doc_ds)} documents")
                
                # Initialize embedder
                embedder = LocalEmbedder(embedder_name, device="cuda")
                
                # Embed documents and queries
                start_time = time()
                current_doc_ds = embed_dataset(current_doc_ds, embedder, column="passage")
                current_query_ds = embed_dataset(query_ds, embedder, column="question")
                elapsed_time = time() - start_time
                
                # Build FAISS index
                current_doc_ds.add_faiss_index(
                    column='embedding',
                    string_factory='Flat',
                    metric_type=faiss.METRIC_INNER_PRODUCT,
                    batch_size=128,
                )
                
                # Retrieve top-100 chunks/documents
                retrieve_k = 100
                res = current_doc_ds.get_index('embedding').search_batch(
                    np.array(current_query_ds['embedding']), 
                    k=retrieve_k
                )
                
                # Map chunk IDs back to parent document IDs if chunked
                if chunk_size is None:
                    index_to_doc_id = np.array(current_doc_ds['id'])
                else:
                    index_to_doc_id = np.array(current_doc_ds['parent_id'])
                
                retrieved_ids_all = index_to_doc_id[res.total_indices]
                
                # Deduplicate documents (keeping top-100 unique docs)
                retrieved_ids_all = deduplicate_retrieved_docs(retrieved_ids_all, retrieve_k)
                
                # Calculate metrics at different k values
                metrics = {}
                for k in [1, 3, 5, 10]:
                    retrieved_ids = retrieved_ids_all[:, :k]
                    metrics = {
                        **metrics,
                        **get_metrics(retrieved_ids, query_ds, k),
                    }
                
                metrics = {
                    **{k: round(v, 4) for k, v in metrics.items()},
                    "elapsed_time": round(elapsed_time, 1),
                    "num_chunks": len(current_doc_ds),
                }
                
                # Parameters to track
                params = {
                    'embed_model': embedder_name,
                    'rerank_model': rerank_model,
                    'chunked': chunk_size is not None,
                    'chunk_size': chunk_size if chunk_size is not None else 'none',
                    'chunk_overlap': ch
                    unk_overlap if chunk_size is not None else 'none',
                    'faiss_metric': faiss_metric,
                }
                
                # Create run name
                if chunk_size is None:
                    run_name = f"{embedder_name_short}_no-chunking"
                else:
                    run_name = f"{embedder_name_short}_cs{chunk_size}_ov{chunk_overlap}"
                
                # Tags for filtering
                tags = {
                    'experiment_type': 'chunking',
                    'phase': 'exploration',
                    'dataset': 'bioasq-mini',
                    'embedder': embedder_name_short,
                    'chunked': str(chunk_size is not None),
                }
                
                # Log to MLflow
                with tracker.start_run(run_name=run_name, tags=tags):
                    tracker.log_params(params)
                    tracker.log_metrics(metrics)

                print(f"\nResults:")
                print(f"  P@10: {metrics.get('P@10', 0):.4f}")
                print(f"  R@10: {metrics.get('R@10', 0):.4f}")
                print(f"  MRR@10: {metrics.get('MRR@10', 0):.4f}")
                print(f"  Time: {elapsed_time:.1f}s")
                
                # Cleanup
                current_doc_ds.drop_index('embedding')
                del embedder
                del current_doc_ds
                del current_query_ds
                gc.collect()
                torch.cuda.empty_cache()
                
            except Exception as e:
                print(f"\nFailed: {e}")
                import traceback
                traceback.print_exc()
                gc.collect()
                torch.cuda.empty_cache()
                continue

print("\n" + "="*80)
print("All experiments completed!")
print("="*80)


Testing: all-MiniLM-L6-v2 | chunk_size=None | overlap=0


100%|██████████| 219/219 [00:00<00:00, 4905.49it/s]


🏃 View run all-MiniLM-L6-v2_no-chunking at: http://localhost:5000/#/experiments/354692744483317738/runs/82816706e00a45318dbaacf717d0c83f
🧪 View experiment at: http://localhost:5000/#/experiments/354692744483317738

Results:
  P@10: 0.2844
  R@10: 0.3734
  MRR@10: 0.6314
  Time: 35.3s

Testing: all-MiniLM-L6-v2 | chunk_size=128 | overlap=0


Chunking: 100%|██████████| 28001/28001 [00:39<00:00, 705.38it/s]


Created 109865 chunks from 28001 documents


...

🏃 View run bge-large-en-v1.5_cs512_ov50 at: http://localhost:5000/#/experiments/354692744483317738/runs/4191efa517c448ad94dc19dbe2f7c87f
🧪 View experiment at: http://localhost:5000/#/experiments/354692744483317738

Results:
  P@10: 0.3556
  R@10: 0.4745
  MRR@10: 0.7539
  Time: 264.8s

Testing: bge-large-en-v1.5 | chunk_size=512 | overlap=100


Chunking: 100%|██████████| 28001/28001 [00:31<00:00, 900.95it/s]


Created 34965 chunks from 28001 documents


100%|██████████| 274/274 [00:00<00:00, 2100.45it/s]


🏃 View run bge-large-en-v1.5_cs512_ov100 at: http://localhost:5000/#/experiments/354692744483317738/runs/092c99f1073945ca8a0a67d3d16ac7bc
🧪 View experiment at: http://localhost:5000/#/experiments/354692744483317738

Results:
  P@10: 0.3561
  R@10: 0.4748
  MRR@10: 0.7533
  Time: 265.8s

All experiments completed!


In [5]:
# Optional: Quick analysis of results
print("\nView all results in MLflow UI at: http://localhost:5000")
print("\nKey questions to explore:")
print("1. Does chunking improve retrieval performance?")
print("2. What is the optimal chunk size for each embedder?")
print("3. Does overlap help? What's the optimal overlap?")
print("4. How does chunking affect embedding time?")


View all results in MLflow UI at: http://localhost:5000

Key questions to explore:
1. Does chunking improve retrieval performance?
2. What is the optimal chunk size for each embedder?
3. Does overlap help? What's the optimal overlap?
4. How does chunking affect embedding time?
